<a href="https://colab.research.google.com/github/SuperKami32/DynaSys-App/blob/main/Atlas_Engine_0_9_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ======================= AI Portfolio Engine v9.2 (Dynamic Sleeves + GPR) ======================
# Full, runnable, single-file script (Colab/local).
# Major features:
#   • Dynamic sleeve architecture: STOCKS, CRYPTO, LOTTERY — targets discovered each run (no fixed 70/25/5).
#   • Anchored Gaussian Process Regression (GPR) short-horizon signal blended with ML views.
#   • Separate stock-lottery and crypto-lottery; lottery sleeve split by best views.
#   • Within-sleeve optimizers (BL/HRP) + ML views + RL nudges per sleeve.
#   • Min-weight threshold to kill micro-position noise.
#   • Optional execution suggestions: limit buys (basis-point offset) and protective stop for lottery sleeve.
#   • Confirm-to-trade flow with cadence lock, Monte Carlo coaching, Roth integer-only + fractional taxable tickets.
#
# Usage:
#   - Paste into a Colab cell and run, or save as .py and %run.
#   - Optional: run RL trainer first to produce /content/rl_checkpoints/agent_weights.json
#   - In a new cell: from AI_Portfolio_Engine_v9_0 import main; results = main(cadence="weekly")
#
# Notes:
#   - Income fields are not required; decisions are based on DCA inputs and current snapshot.
#   - E*TRADE/broker adapters can replace CONFIG["initial"] later.
#
import importlib, subprocess, sys, os, json, warnings
from typing import Dict, List
def ensure(pkg, import_name=None):
    mod = import_name or pkg
    try:
        importlib.import_module(mod)
    except ModuleNotFoundError:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=False)
        importlib.invalidate_caches()

# Pin stable versions
for pip_name, imp in [
    ("yfinance==0.2.52","yfinance"),
    ("pandas==2.2.2","pandas"),
    ("numpy==1.26.4","numpy"),
    ("scikit-learn==1.5.1","sklearn"),
    ("PyPortfolioOpt==1.5.5","pypfopt"),
    ("matplotlib==3.9.0","matplotlib"),
]:
    ensure(pip_name, imp)

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from pypfopt import risk_models, black_litterman
from pypfopt.hierarchical_portfolio import HRPOpt
from enum import Enum
from RL_Portfolio_Trainer_v7_0 import train_rl_agent
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

# ---------------------------- Paths & Run Folder ----------------------------
SAVE_DIR = "/content/rl_checkpoints"
os.makedirs(SAVE_DIR, exist_ok=True)
RUN_ID  = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
RUN_DIR = f"/content/run_{RUN_ID}"
os.makedirs(RUN_DIR, exist_ok=True)
print(f"[Logger] Run folder → {RUN_DIR}")

# ---------------------------- Config ----------------------------
CONFIG = {
    "switches": {
        "use_yfinance": True,
        "use_selector_engine": True,
        "use_ml_models": True,
        "use_optimizer": "AUTO",   # "AUTO" | "BL" | "HRP" | "OFF"
        "use_monte_carlo": True,
        "use_rl_hook": True,
        "export_trade_csv": True,
        "use_balance_sync": False
    },

    # Sleeve discovery (no fixed targets). Bounds keep sanity.
    "sleeves": {
        "bounds":  {"stocks": (0.40, 0.90), "crypto": (0.05, 0.40), "lottery": (0.00, 0.10)},
        "per_ticker_cap": {"stocks": 0.30, "crypto": 0.30, "lottery": 1.00},
        "min_weight_threshold": 0.005,     # drop <0.5%
        "softmax_temp": 2.0                # higher temp = more even sleeves
    },

    # Universes (Core pistons + majors; selector can draft alternates)
    "stock_universe": [
      "QQQM","VTI","AVUV","AVDV","SCHD","O","JEPI","VGT","DGRO","VIG","DIVO","JEPQ","TLT","BIL",
      "NVDA","AAPL","MSFT","TSLA","AMZN","TQQQ","SOXL","QYLD","XYLD"],
      "crypto_universe": [
      "BTC-USD","ETH-USD","SOL-USD","LINK-USD","ADA-USD","DOT-USD","AVAX-USD","UNI-USD","AAVE-USD"],
      "stock_lottery_tickers": ["ARKK","TQQQ","SOXL"],
      "crypto_lottery_tickers": ["DOGE-USD","PTB-USD"],


    # Buckets
    "asset_classes": {
        "QQQM":"growth","VTI":"growth","AVUV":"growth","AVDV":"growth","VGT":"growth",
        "SCHD":"income","O":"income","JEPI":"income","DGRO":"income","VIG":"income","DIVO":"income","JEPQ":"income","TLT":"income","BIL":"income",
        "ARKK":"growth",
        "BTC-USD":"growth","ETH-USD":"growth","SOL-USD":"growth","LINK-USD":"growth","DOGE-USD":"growth"
    },

    "market_proxy": ["VTI"],

    # Execution hints
    "execution": {
        "use_limit_buys": True,          # suggest limit buys with offset (not mandatory)
        "limit_offset_bps": 15,           # 15 bps below last price → limit = price * (1 - 0.0015)
        "use_protective_stop_lottery": True,
        "stop_loss_pct_lottery": 0.15     # 15% stop only for lottery sleeve tickers
    },

    # Accounts & DCA snapshot
    "initial": {
        "taxable_total_value": 5200.0,
        "roth_total_value":    3500.0,
        "taxable_cash": 0.0,
        "roth_cash":    0.0,
        "taxable_positions": {"VTI": 1, "SCHD": 1},
        "roth_positions":    {"QQQM": 1, "AVUV": 1, "AVDV": 1}
    },
    "dca": {"assume_today": True, "amount_taxable": 260.0, "amount_roth": 140.0},

    # Tax
    "tax_rates": {"qualified_div": 0.15, "ordinary_income": 0.24},
    "asset_tax_types": {"O":"ordinary","JEPI":"ordinary","JEPQ":"ordinary","XYLD":"ordinary","TLT":"ordinary"},

    # Optimizer knobs
    "optimizer_bl_tau": 0.05,
    "optimizer_view_edge_scale": 0.60,

    # Monte Carlo
    "goal_amount": 1_000_000,
    "goal_end_date": "2035-12-31",
    "monte_carlo_sims": 8000,

    # RL
    "rl_policy_path": os.path.join(SAVE_DIR, "agent_weights.json"),
    "weights_history_path": os.path.join(SAVE_DIR, "last_weights.json"),

    # Data
    "data_start_date": "2015-01-01",

    "macro_policy": {
      "stance": "neutral",   # "easing", "tightening", or "neutral"
      "market": 0.0,         # optional, sentiment-style tilt (-1.0 to +1.0)
      "vol": 0.0,            # optional, perceived volatility signal (-1.0 calm, +1.0 stress)
      "policy": 0.0          # optional, direct policy stance (-1.0 hawkish, +1.0 dovish)
    }
}


with open(os.path.join(RUN_DIR, "config_snapshot.json"), "w") as f:
    json.dump(CONFIG, f, indent=2, default=str)

# ---------------------------- Utilities ----------------------------
_INFO_CACHE: Dict[str, dict] = {}

def classify_bucket(ticker: str) -> str:
    return CONFIG["asset_classes"].get(ticker.upper(), "growth")

def normalize_weights(w: Dict[str, float]) -> Dict[str, float]:
    s = sum(max(v,0.0) for v in w.values()) or 1.0
    return {k: max(0.0,float(v))/s for k,v in w.items()}

def min_weight_prune(weights: Dict[str,float], threshold: float) -> Dict[str,float]:
    pruned = {t: (w if w >= threshold else 0.0) for t,w in weights.items()}
    return normalize_weights(pruned)

def clip_cap(weights: Dict[str,float], cap: float) -> Dict[str,float]:
    if cap is None or cap >= 0.999: return normalize_weights(weights)
    w2 = {k: min(v, cap) for k,v in weights.items()}
    return normalize_weights(w2)

def fetch_prices(tickers: List[str], start: str) -> pd.DataFrame:
    if CONFIG["switches"]["use_yfinance"]:
        df_full = yf.download(tickers, start=start, auto_adjust=True, progress=False,
                              group_by="ticker", threads=True)
        closers = []
        for t in tickers:
            try:
                if isinstance(df_full.columns, pd.MultiIndex):
                    s = df_full[t]["Close"].rename(t)
                else:
                    s = df_full["Close"].rename(t)
                if not s.empty: closers.append(s)
            except Exception:
                warnings.warn(f"Skipping {t}")
        if not closers:
            return pd.DataFrame()
        return pd.concat(closers, axis=1).ffill().dropna(how="all")
    # synthetic fallback
    idx = pd.bdate_range(start=start, end=pd.Timestamp.today())
    df = pd.DataFrame(index=idx)
    rng = np.random.default_rng(42)
    for t in tickers:
        mu, sigma = (0.06, 0.12) if classify_bucket(t) == "income" else (0.07, 0.18)
        daily_mu = mu/252; daily_sd = sigma/np.sqrt(252)
        r = rng.normal(daily_mu, daily_sd, len(idx))
        df[t] = 100 * np.cumprod(1 + r)
    return df

def _get_etf_info(t) -> dict:
    if t in _INFO_CACHE: return _INFO_CACHE[t]
    try: info = yf.Ticker(t).info
    except Exception: info = {}
    _INFO_CACHE[t] = info or {}
    return _INFO_CACHE[t]

def _avg_dollar_vol_3m(t) -> float:
    try:
        h = yf.Ticker(t).history(period="3mo", interval="1d", auto_adjust=False)
        if h.empty: return np.nan
        dv = (h["Close"] * h["Volume"]).dropna()
        return float(dv.mean()) if not dv.empty else np.nan
    except Exception:
        return np.nan

# ---------------------------- GPR (Anchored) ----------------------------
def anchored_gpr_signal(series: pd.Series, forecast_len=10, smooth=40, sigma=0.15) -> float:
    s = series.dropna().values
    if s.shape[0] < smooth: return 0.0
    X = np.arange(len(s)).reshape(-1, 1)
    y = np.log(s)
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=smooth, length_scale_bounds=(1, 1e3))
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=sigma**2, normalize_y=True)
    gpr.fit(X, y)
    Xf = np.arange(len(s), len(s)+forecast_len).reshape(-1,1)
    y_pred, y_std = gpr.predict(Xf, return_std=True)
    slope = y_pred[-1] - y_pred[0]
    confidence = 1.0 - np.clip(np.mean(y_std), 0, 1)
    return float(np.tanh(slope)) * confidence  # [-1,1]

# ---------------------------- ML Views ----------------------------
def make_features(prices: pd.DataFrame, market_series: pd.Series):
    feats = {}
    for t in prices.columns:
        s = prices[t].dropna()
        if s.shape[0] < 252: continue
        df = pd.DataFrame(index=s.index)
        df["ret_21"]      = s.pct_change(21)
        df["ret_63"]      = s.pct_change(63)
        df["vol_21"]      = s.pct_change().rolling(21).std()
        df["ma_200_dist"] = s/s.rolling(200).mean() - 1
        m = market_series.reindex(df.index).ffill()
        df["mkt_ret_21"]  = m.pct_change(21)
        df["target"]      = (s.shift(-21)/s - 1 > 0).astype(int)
        feats[t] = df.dropna()
    return feats

def train_models(features: Dict[str, pd.DataFrame]) -> Dict[str, tuple]:
    models = {}
    for t, df in features.items():
        X, y = df.drop(columns=["target"]), df["target"]
        if len(y.unique()) < 2 or len(y) < 200: continue
        rf = CalibratedClassifierCV(RandomForestClassifier(n_estimators=300, random_state=42), cv=3)
        lr = CalibratedClassifierCV(LogisticRegression(max_iter=300), cv=3)
        rf.fit(X, y); lr.fit(X, y)
        models[t] = (rf, lr)
    return models

def get_model_views(models: Dict[str, tuple], features: Dict[str, pd.DataFrame]) -> Dict[str, float]:
    views = {}
    for t, mdl in models.items():
        rf, lr = mdl
        Xtail = features[t].drop(columns=["target"]).iloc[-1:].values
        if Xtail.shape[0]:
            p_rf = rf.predict_proba(Xtail)[0, 1]
            p_lr = lr.predict_proba(Xtail)[0, 1]
            views[t] = float(0.6*p_rf + 0.4*p_lr)  # [0,1]
    return views

def blend_views_with_gpr(prices: pd.DataFrame, views: Dict[str,float], gpr_weight=0.35) -> Dict[str,float]:
    out = {}
    for t in prices.columns:
        v = views.get(t, 0.5)
        g = (anchored_gpr_signal(prices[t]) + 1)/2  # map [-1,1] -> [0,1]
        out[t] = float((1-gpr_weight)*v + gpr_weight*g)
    return out

# ---------------------------- Candidate Screening ----------------------------
def screen_candidates(candidates: List[str], prices: pd.DataFrame) -> List[str]:
    screened = []
    for t in candidates:
        if t not in prices.columns or prices[t].dropna().shape[0] < 252*3: continue
        info = _get_etf_info(t) if CONFIG["switches"]["use_yfinance"] else {}
        expense = info.get("expenseRatio") or info.get("annualReportExpenseRatio") or 0.003
        aum     = info.get("totalAssets") or info.get("totalAssetsRaw") or 2_000_000_000
        adv     = _avg_dollar_vol_3m(t) if CONFIG["switches"]["use_yfinance"] else 20_000_000
        if expense <= 0.0035 and aum >= 1_000_000_000 and (np.isnan(adv) or adv >= 10_000_000):
            screened.append(t)
    return screened

# ---------------------------- Optimizers ----------------------------
def _bl_weights(universe, prices, views_prob=None, tau=0.05, view_scale=0.60):
    rets = prices[universe].pct_change().dropna()
    if rets.empty: return {t: 1/len(universe) for t in universe}
    S = risk_models.sample_cov(rets, frequency=252)
    pi = rets.mean() * 252
    Q = {}
    if views_prob:
        for t, p in views_prob.items():
            if t in universe:
                edge = (p - 0.5) * 2.0  # [-1,1]
                Q[t] = float(pi.get(t, 0)) + view_scale * edge * abs(float(pi.get(t, 0)) + 1e-3)
    if Q:
        P = pd.DataFrame(0.0, index=range(len(Q)), columns=universe)
        qv = []
        for i, (t, v) in enumerate(Q.items()):
            P.loc[i, t] = 1.0; qv.append(v)
        bl = black_litterman.BlackLittermanModel(S, pi=pi, P=P.values, Q=np.array(qv), tau=tau)
        w = pd.Series(bl.bl_weights(), dtype=float).reindex(universe).fillna(0)
        w = w.clip(lower=0).to_dict()
        s = sum(w.values()) or 1.0
        return {k: v/s for k, v in w.items()}
    hrp = HRPOpt(rets)
    return hrp.optimize()

def _hrp_weights(universe, prices):
    rets = prices[universe].pct_change().dropna()
    if rets.empty: return {t: 1/len(universe) for t in universe}
    hrp = HRPOpt(rets); return hrp.optimize()

def get_optimizer_weights(mode, universe, prices, views_prob):
    mode = (mode or "AUTO").upper()
    if mode == "BL":
        return _bl_weights(universe, prices, views_prob, tau=CONFIG["optimizer_bl_tau"],
                           view_scale=CONFIG["optimizer_view_edge_scale"])
    if mode == "HRP":
        return _hrp_weights(universe, prices)
    if mode == "OFF":
        return {t: 1/len(universe) for t in universe}
    try:
        w = _bl_weights(universe, prices, views_prob, tau=CONFIG["optimizer_bl_tau"],
                        view_scale=CONFIG["optimizer_view_edge_scale"])
        if sum(w.values()) > 0: return w
    except Exception:
        pass
    return _hrp_weights(universe, prices)

# ---------------------------- RL Nudges ----------------------------
def apply_rl_nudges(weights: dict, path=CONFIG["rl_policy_path"], blend=0.15):
    try:
        nudges = json.load(open(path,"r"))
    except Exception:
        return weights
    out = {}
    for t,w in weights.items():
        delta = float(nudges.get(t, 0.0))
        out[t] = max(0.0, w * (1.0 + blend * delta))
    return normalize_weights(out)

# ---------------------------- Regime & Sleeve Targets (Dynamic) ----------------------------
def get_regime(prices: pd.DataFrame, proxy: str):
    s = prices[proxy].dropna()
    if s.empty or len(s) < 200:
        return {"name":"sideways","confidence":0.33}
    ma50  = s.rolling(50).mean(); ma200 = s.rolling(200).mean()
    spread = float((ma50.iloc[-1] - ma200.iloc[-1]) / (ma200.iloc[-1] + 1e-9))
    vol = float(s.pct_change().rolling(21).std().iloc[-1])
    if spread > 0.01:
        name = "bull"; conf = np.clip(0.5 + spread - 0.5*vol, 0.1, 0.95)
    elif spread < -0.01:
        name = "bear"; conf = np.clip(0.5 + (-spread) - 0.5*vol, 0.1, 0.95)
    else:
        name = "sideways"; conf = 0.33
    return {"name":name, "confidence": float(conf)}

def softmax(x):
    x = np.array(x, dtype=float)
    x = x - np.max(x)
    e = np.exp(x)
    return (e / np.sum(e)).tolist()

def compute_dynamic_sleeve_targets(prices: pd.DataFrame,
                                   views_blend: Dict[str,float],
                                   regime: dict) -> Dict[str,float]:
    sleeves = ["stocks","crypto","lottery"]
    bounds  = CONFIG["sleeves"]["bounds"]
    temp    = CONFIG["sleeves"]["softmax_temp"]

    stock_tickers  = [t for t in CONFIG["stock_universe"]  if t in prices.columns]
    crypto_tickers = [t for t in CONFIG["crypto_universe"] if t in prices.columns]
    lot_tickers    = [t for t in (CONFIG["stock_lottery_tickers"] + CONFIG["crypto_lottery_tickers"]) if t in prices.columns]

    def sleeve_score(tickers):
        if not tickers: return 0.0
        v = np.mean([views_blend.get(t,0.5) for t in tickers])
        # momentum boost
        mom = 0.0
        try:
            r = prices[tickers].pct_change(21).iloc[-1].dropna()
            mom = float(np.tanh(r.mean()*5))
        except Exception:
            pass
        return float(0.7*v + 0.3*(mom+0.5))

    scores = [
        sleeve_score(stock_tickers),
        sleeve_score(crypto_tickers),
        sleeve_score(lot_tickers) * 0.8,
    ]

    # Regime nudges
    if regime["name"]=="bull":
        scores[0] += 0.05; scores[1] += 0.03
    elif regime["name"]=="bear":
        scores[0] -= 0.05; scores[1] -= 0.03

    # Macro policy nudges
    macro = CONFIG.get("macro_policy", {}).get("stance", None)
    if macro == "easing":
        scores[0] += 0.03   # tilt stocks
        scores[1] += 0.02   # tilt crypto
    elif macro == "tightening":
        scores[0] -= 0.03
        scores[1] -= 0.02

    # Softmax with temperature
    probs = np.array(softmax(np.array(scores)/max(1e-6, temp)))
    # Clamp to bounds and renormalize
    targets = {"stocks": float(np.clip(probs[0], *bounds["stocks"])),
               "crypto": float(np.clip(probs[1], *bounds["crypto"])),
               "lottery": float(np.clip(probs[2], *bounds["lottery"]))}
    s = sum(targets.values())
    for k in targets: targets[k] /= s
    return targets


# ---------------------------- Sleeve Optimizations ----------------------------
def sleeve_optimize(universe: List[str], prices: pd.DataFrame, views: Dict[str,float]) -> Dict[str,float]:
    mode = CONFIG["switches"]["use_optimizer"]
    u = [t for t in universe if t in prices.columns]
    if not u: return {}
    weights = get_optimizer_weights(mode, u, prices, views)
    # --- Diversification guardrail ---
    if sum(weights.values()) > 0.30:  # sleeve got >30% of total portfolio
        if len([w for w in weights.values() if w > 0]) < 2:
            # force split between top 2 tickers
            top2 = sorted(weights.items(), key=lambda x: x[1], reverse=True)[:2]
            total = sum(v for _, v in top2) or 1.0
            weights = {k: v/total for k, v in top2}

    return weights

def build_lottery_weights(stock_lot: List[str], crypto_lot: List[str],
                          prices: pd.DataFrame, views: Dict[str,float]) -> Dict[str,float]:
    candidates = [t for t in (stock_lot + crypto_lot) if t in prices.columns]
    if not candidates: return {}
    scored = sorted(candidates, key=lambda t: views.get(t, 0.5), reverse=True)
    top = scored[:1]  # single-slot lottery
    return normalize_weights({t: 1/len(top) for t in top})

# ---------------------------- Monte Carlo & Coaching ----------------------------
def run_monte_carlo(initial_value: float, months: int, monthly_dca: float,
                    portfolio_returns: pd.Series, sims: int):
    mu, sigma = float(portfolio_returns.mean()), float(portfolio_returns.std())
    rng = np.random.default_rng(123)
    sim = rng.normal(mu, sigma, (months, sims))
    finals = np.full(sims, initial_value, dtype=float)
    for i in range(months):
        finals = finals * (1 + sim[i, :]) + monthly_dca
    return finals

def print_probability_gauge(prob: float):
    bars = 34; filled = int(np.clip(prob,0,1)*bars)
    color_icon = "🟢" if prob>=0.75 else ("🟡" if prob>=0.60 else "🔴")
    print("\n=== Probability Gauge ===")
    print(f"[{color_icon}{'█'*filled}{'░'*(bars-filled)}] {prob:.1%}")

def goal_delta_coaching(prob: float, p50: float, months: int, goal: float):
    status = "ON TRACK" if prob>=0.75 else ("WATCH" if prob>=0.60 else "BEHIND")
    print("\n=== Goal Delta Coaching ===")
    if status=="BEHIND":
        short = max(0.0, goal - p50)
        add_per_month = short/months if months>0 else 0.0
        print("⚠️ Behind: increase DCA or accept higher risk.")
        print(f"    Add ≈ ${add_per_month:,.0f}/month to reach median path to ${goal:,.0f}.")
    elif status=="WATCH":
        print("⚠️ Watch: small DCA or sleeve tilt could push odds above 75%.")
    else:
        print("✅ On track: consider nudging toward income to lock progress.")

# ---------------------------- Tax-aware mapping ----------------------------
def get_tax_aware_mapping(global_w: Dict[str, float]) -> Dict[str, Dict[str, float]]:
    roth_w, taxable_w = {}, {}
    roth_prefers = {"income"}
    for t,w in global_w.items():
        if CONFIG["asset_tax_types"].get(t)=="ordinary" or classify_bucket(t) in roth_prefers:
            roth_w[t] = w
        else:
            taxable_w[t] = w
    if not roth_w:    roth_w    = taxable_w.copy() or global_w.copy()
    if not taxable_w: taxable_w = roth_w.copy() or global_w.copy()
    return {"roth": normalize_weights(roth_w), "taxable": normalize_weights(taxable_w)}

# ---------------------------- Ticketing (with execution hints) ----------------------------
def build_trade_tickets(latest_prices: Dict[str, float], sleeve_w: Dict[str, Dict[str, float]], cash: Dict[str, float],
                        lottery_names: List[str]):
    fp   = 3
    min_d = 25.0
    taxable_orders, roth_orders = [], []
    limit_on  = CONFIG["execution"]["use_limit_buys"]
    lim_bps   = CONFIG["execution"]["limit_offset_bps"]/10000.0
    use_stop  = CONFIG["execution"]["use_protective_stop_lottery"]
    stop_pct  = CONFIG["execution"]["stop_loss_pct_lottery"]

    def mk_order(t, shares, price):
        order = {"ticker": t, "shares": shares, "dollars": round(shares*price,2)}
        if limit_on:
            order["order_type"] = "limit"
            order["limit_price"] = round(price*(1.0 - lim_bps), 4)
        else:
            order["order_type"] = "market"
        if use_stop and t in lottery_names:
            order["protective_stop"] = round(price*(1.0 - stop_pct), 4)
        return order

    # Taxable = fractional
    if cash["taxable"] > min_d and sleeve_w["taxable"]:
        for t, w in sleeve_w["taxable"].items():
            p = float(latest_prices.get(t, 0))
            if p <= 0: continue
            dollars = cash["taxable"] * w
            if dollars >= min_d:
                shares = round(dollars / p, fp)
                if shares > 0:
                    taxable_orders.append(mk_order(t, shares, p))

    # Roth = integer-only greedy
    if cash["roth"] > min_d and sleeve_w["roth"]:
        budget = float(cash["roth"]); target_w = sleeve_w["roth"]
        prices_now = {t: float(latest_prices.get(t, 0)) for t in target_w if float(latest_prices.get(t, 0)) > 0}
        shares_to_buy = {t: 0 for t in prices_now}; spent = 0.0
        for _ in range(100000):
            under = {t: (budget*target_w.get(t,0) - shares_to_buy[t]*prices_now[t]) for t in prices_now}
            if not under: break
            best = max(under, key=under.get)
            price = prices_now[best]
            if price < min_d or spent + price > budget: break
            shares_to_buy[best] += 1; spent += price
        for t, n in shares_to_buy.items():
            if n > 0:
                roth_orders.append(mk_order(t, int(n), prices_now[t]))
    return {"taxable": taxable_orders, "roth": roth_orders}

# ---------------------------- Confirm-to-trade State Machine ----------------------------
class State(Enum): IDLE=0; PROPOSE=1; AWAIT_CONFIRM=2; EXECUTING=3; COOLDOWN=4
STATE = State.IDLE
LOCK_UNTIL = None  # pd.Timestamp

def propose_trades(tickets, cadence="weekly"):
    global STATE, LOCK_UNTIL
    now = pd.Timestamp.utcnow()
    if STATE in (State.COOLDOWN, State.AWAIT_CONFIRM) and LOCK_UNTIL is not None and now < LOCK_UNTIL:
        print("⏳ Locked until", LOCK_UNTIL); return None
    STATE = State.PROPOSE
    print("\n🧾 Proposed trades:", json.dumps(tickets, indent=2))
    STATE = State.AWAIT_CONFIRM
    days = {"daily":1, "weekly":7, "monthly":30}.get(cadence, 1)
    LOCK_UNTIL = now + pd.Timedelta(days=days)
    return {"expires_at": LOCK_UNTIL.isoformat()}

def confirm_execute(place_order_fn, tickets):
    global STATE
    if STATE != State.AWAIT_CONFIRM:
        print("No pending proposal."); return
    STATE = State.EXECUTING
    try:
        place_order_fn(tickets)
        STATE = State.COOLDOWN
        print("✅ Orders placed. Cooldown in effect.")
    except Exception as e:
        STATE = State.IDLE
        print(f"❌ Execution failed: {e}")

def cancel_proposal():
    global STATE
    if STATE == State.AWAIT_CONFIRM: STATE = State.IDLE
# ================= Broker Adapters (stubs) =================
def fetch_etrade_trade_history(account_id=None, count=100):
    """
    Pull last N trades from E*TRADE API.
    Return list of dicts with standard schema:
    date, symbol, type, price, quantity, cost, fee, pl
    """
    # TODO: wire up to real E*TRADE API
    return [
        {"date": "2025-09-01", "symbol": "VTI", "type": "BUY",
         "price": 250.0, "quantity": 2, "cost": 500.0, "fee": 0.0, "pl": 0.0}
    ]

def fetch_kraken_trade_history(count=100):
    """
    Pull last N trades from Kraken API.
    Return list of dicts with standard schema:
    date, pair, type, price, quantity, cost, fee, pl
    """
    # TODO: wire up to real Kraken API
    return [
        {"date": "2025-09-05", "pair": "BTC-USD", "type": "BUY",
         "price": 60000.0, "quantity": 0.01, "cost": 600.0, "fee": 1.0, "pl": 0.0}
    ]

def place_order_etrade(tickets):
    """
    Place trades with E*TRADE.
    For now, just print. Replace with API calls later.
    """
    for acct, orders in tickets.items():
        for order in orders:
            print(f"[E*TRADE] {acct.upper()} placing {order['order_type']} "
                  f"order for {order['shares']}x {order['ticker']} "
                  f"(${order.get('limit_price') or order['dollars']})")

def place_order_kraken(tickets):
    """
    Place trades with Kraken.
    For now, just print. Replace with API calls later.
    """
    for acct, orders in tickets.items():
        for order in orders:
            print(f"[Kraken] {acct.upper()} placing {order['order_type']} "
                  f"order for {order['shares']} {order['ticker']} "
                  f"(stop {order.get('protective_stop')})")

# ================= Trade History Reflection =================
def analyze_trade_history(trades, lookback_days=90, console=True):
    """
    Analyze recent trade history and return a dict for reflection.
    Optionally print a coach-style recap if console=True.
    """
    import pandas as pd, numpy as np
    if not trades:
        if console:
            print("\n=== Trade History Reflection ===\nNo trades found in history.")
        return {"message": "No trades found in history."}

    df = pd.DataFrame(trades).copy()
    if "symbol" not in df.columns and "pair" in df.columns:
        df["symbol"] = df["pair"]

    # Parse dates
    df["date"] = pd.to_datetime(df["date"], unit="s", errors="coerce") \
                   .fillna(pd.to_datetime(df["date"], errors="coerce"))
    cutoff = pd.Timestamp.utcnow() - pd.Timedelta(days=lookback_days)
    recent = df[df["date"] >= cutoff].copy()
    if recent.empty:
        if console:
            print(f"\n=== Trade History Reflection ===\nNo trades in the last {lookback_days} days.")
        return {"message": f"No trades in the last {lookback_days} days."}

    realized_pl = float(recent["pl"].sum() if "pl" in recent else 0.0)
    total_fees  = float(recent["fee"].sum() if "fee" in recent else 0.0)
    wins = int((recent["pl"] > 0).sum() if "pl" in recent else 0)
    losses = int((recent["pl"] < 0).sum() if "pl" in recent else 0)
    total = max(1, wins + losses)
    win_rate = wins / total

    # Average hold
    avg_hold_days = None
    try:
        hold_times = []
        for sym in recent["symbol"].unique():
            sym_trades = recent[recent["symbol"] == sym].sort_values("date")
            stack = []
            for _, row in sym_trades.iterrows():
                if row["type"].upper() == "BUY":
                    stack.append(row["date"])
                elif row["type"].upper() == "SELL" and stack:
                    buy_date = stack.pop(0)
                    hold_times.append((row["date"] - buy_date).days)
        if hold_times:
            avg_hold_days = float(np.mean(hold_times))
    except Exception:
        avg_hold_days = None

    reflection = {
        "lookback_days": lookback_days,
        "realized_pl": realized_pl,
        "total_fees": total_fees,
        "net_pl": realized_pl - total_fees,
        "wins": wins,
        "losses": losses,
        "win_rate": float(win_rate),
        "avg_hold_days": avg_hold_days,
        "best": None,
        "worst": None,
        "sleeve_attribution": None
    }

    # Best/Worst
    if "symbol" in recent:
        grp = recent.groupby("symbol")["pl"].sum().sort_values(ascending=False)
        if not grp.empty:
            reflection["best"]  = {"symbol": grp.index[0], "pl": float(grp.values[0])}
            reflection["worst"] = {"symbol": grp.index[-1], "pl": float(grp.values[-1])}

    # Sleeve attribution
    try:
        def classify_sleeve(sym: str):
            if sym in CONFIG["crypto_universe"]: return "crypto"
            if sym in CONFIG["stock_lottery_tickers"] or sym in CONFIG["crypto_lottery_tickers"]: return "lottery"
            if CONFIG["asset_classes"].get(sym) == "income": return "income"
            return "growth"
        recent["sleeve"] = recent["symbol"].map(classify_sleeve)
        sleeve_grp = recent.groupby("sleeve")["pl"].sum().to_dict()
        reflection["sleeve_attribution"] = {k: float(v) for k,v in sleeve_grp.items()}
    except Exception:
        reflection["sleeve_attribution"] = None

    # === Console Recap ===
    if console:
        bars = 34
        filled = int(np.clip(win_rate,0,1)*bars)
        color_icon = "🟢" if win_rate>=0.65 else ("🟡" if win_rate>=0.50 else "🔴")
        print("\n=== Trade History Reflection ===")
        print(f"[{color_icon}{'█'*filled}{'░'*(bars-filled)}] {win_rate:.1%} win rate over last {lookback_days} days")
        print(f"• Realized P/L: ${realized_pl:,.2f} (Net: ${realized_pl-total_fees:,.2f} after ${total_fees:,.2f} fees)")
        print(f"• Trades: {total} (Wins: {wins}, Losses: {losses})")
        if avg_hold_days is not None:
            print(f"• Avg Hold: {avg_hold_days:.1f} days")
        if reflection["best"]: print(f"• Best: {reflection['best']['symbol']} +${reflection['best']['pl']:,.2f}")
        if reflection["worst"]: print(f"• Worst: {reflection['worst']['symbol']} ${reflection['worst']['pl']:,.2f}")
        if reflection["sleeve_attribution"]:
            print("• Sleeve Attribution:")
            for s,v in reflection["sleeve_attribution"].items():
                print(f"   {s}: ${v:,.2f}")

    return reflection


def retrain_agent(run_sweep=True):
    """
    Kick off an RL retraining cycle and return paths to new nudges + eval summary.
    """
    try:
        res = train_rl_agent(run_sweep_flag=run_sweep, export_nudges_flag=True)
        return {"status": "ok", "paths": res}
    except Exception as e:
        return {"status": "error", "error": str(e)}

# ---------------------------- Main ----------------------------
# ================= Logging & RL Eval =================
LOG_DB = "/content/portfolio_log.json"

def append_run_log(entry: dict, path=LOG_DB):
    import json, os
    all_logs = []
    if os.path.exists(path):
        with open(path, "r") as f:
            try: all_logs = json.load(f)
            except: all_logs = []
    all_logs.append(entry)
    with open(path, "w") as f:
        json.dump(all_logs, f, indent=2, default=str)

def load_rl_eval_summary(path=os.path.join(SAVE_DIR, "evaluation_summary.json")):
    import json
    try:
        with open(path, "r") as f:
            return json.load(f)
    except Exception:
        return {"message": "No RL eval summary found."}
def last_business_day(d):
    d = pd.to_datetime(d); return d - pd.offsets.BDay(0)

def main(cadence="weekly"):
    sleeves_cfg = CONFIG["sleeves"]
    # Assemble universes
    stock_u = list(dict.fromkeys(CONFIG["stock_universe"] + CONFIG["stock_lottery_tickers"]))
    crypto_u = list(dict.fromkeys(CONFIG["crypto_universe"] + CONFIG["crypto_lottery_tickers"]))
    base_universe = sorted(set(stock_u + crypto_u))

    prices = fetch_prices(base_universe, CONFIG["data_start_date"]).dropna(how="all")
    if prices.empty:
        raise RuntimeError("Price fetch returned empty. Check tickers or network.")
    latest_px = {t: float(prices[t].iloc[-1]) for t in prices.columns}

    # Selector engine (stocks only) – drafts alternates by cost/liquidity
    if CONFIG["switches"]["use_selector_engine"] and CONFIG["switches"]["use_yfinance"]:
        filtered = screen_candidates(CONFIG["stock_universe"], prices)
        if filtered:
            CONFIG["stock_universe"] = sorted(set(filtered) | set([t for t in CONFIG["stock_universe"] if t in filtered]))

    # ML views + GPR blend
    views = {}
    if CONFIG["switches"]["use_ml_models"]:
        feats = make_features(prices, prices[CONFIG["market_proxy"][0]])
        models = train_models(feats)
        views = get_model_views(models, feats)
    blended_views = blend_views_with_gpr(prices, views, gpr_weight=0.35)

    # Regime + dynamic sleeve targets
    regime = get_regime(prices, CONFIG["market_proxy"][0])
    sleeve_targets = compute_dynamic_sleeve_targets(prices, blended_views, regime)

    # Optimize within sleeves (exclude lottery tickers)
    stock_core  = [t for t in CONFIG["stock_universe"]  if t not in CONFIG["stock_lottery_tickers"]]
    crypto_core = [t for t in CONFIG["crypto_universe"] if t not in CONFIG["crypto_lottery_tickers"]]

    w_stocks = sleeve_optimize(stock_core, prices, blended_views)
    w_crypto = sleeve_optimize(crypto_core, prices, blended_views)

    # RL nudges per sleeve
    if CONFIG["switches"]["use_rl_hook"]:
        w_stocks = apply_rl_nudges(w_stocks, path=CONFIG["rl_policy_path"], blend=0.15)
        w_crypto = apply_rl_nudges(w_crypto, path=CONFIG["rl_policy_path"], blend=0.15)

    # Caps & prune
    w_stocks = clip_cap(w_stocks, sleeves_cfg["per_ticker_cap"]["stocks"])
    w_crypto = clip_cap(w_crypto, sleeves_cfg["per_ticker_cap"]["crypto"])
    w_stocks = min_weight_prune(w_stocks, sleeves_cfg["min_weight_threshold"])
    w_crypto = min_weight_prune(w_crypto, sleeves_cfg["min_weight_threshold"])

    # Lottery per sleeve
    w_lottery = build_lottery_weights(CONFIG["stock_lottery_tickers"], CONFIG["crypto_lottery_tickers"], prices, blended_views)

    # Compose final portfolio from sleeves
    portfolio = {}
    for t,w in w_stocks.items():
        portfolio[t] = portfolio.get(t,0.0) + sleeve_targets["stocks"] * w
    for t,w in w_crypto.items():
        portfolio[t] = portfolio.get(t,0.0) + sleeve_targets["crypto"] * w
    for t,w in w_lottery.items():
        portfolio[t] = portfolio.get(t,0.0) + sleeve_targets["lottery"] * w
    portfolio = normalize_weights(portfolio)

    # Map to accounts
    sleeve_w = get_tax_aware_mapping(portfolio)

    # Tickets with execution hints
    cash = {"taxable": CONFIG["dca"]["amount_taxable"], "roth": CONFIG["dca"]["amount_roth"]}
    lottery_names = list(w_lottery.keys())
    tickets = build_trade_tickets(latest_px, sleeve_w, cash, lottery_names)

    # Monte Carlo
    months = int((pd.to_datetime(CONFIG["goal_end_date"]) - last_business_day(pd.Timestamp.today())).days/30.44)
    initial_value = CONFIG["initial"]["taxable_total_value"] + CONFIG["initial"]["roth_total_value"]
    monthly_dca = CONFIG["dca"]["amount_taxable"] + CONFIG["dca"]["amount_roth"]
    cols = [t for t in portfolio if t in prices.columns]
    port_rets = prices[cols].pct_change().dropna().dot(pd.Series(portfolio).reindex(cols).fillna(0))
    finals = run_monte_carlo(initial_value, months, monthly_dca, port_rets, CONFIG["monte_carlo_sims"]) if CONFIG["switches"]["use_monte_carlo"] else np.array([])
    prob = float((finals >= CONFIG["goal_amount"]).mean()) if finals.size else 0.0
    p50  = float(np.median(finals)) if finals.size else initial_value
    print_probability_gauge(prob); goal_delta_coaching(prob, p50, months, CONFIG["goal_amount"])

    # Export CSV
    if CONFIG["switches"]["export_trade_csv"]:
        out_path = os.path.join(RUN_DIR, "proposed_trades.csv")
        rows = [("taxable",o["ticker"],o["shares"],o["dollars"],o.get("order_type"),o.get("limit_price"),o.get("protective_stop")) for o in tickets["taxable"]] + \
               [("roth",   o["ticker"],o["shares"],o["dollars"],o.get("order_type"),o.get("limit_price"),o.get("protective_stop")) for o in tickets["roth"]]
        pd.DataFrame(rows, columns=["account","ticker","shares","dollars","order_type","limit_price","protective_stop"]).to_csv(out_path, index=False)
        print(f"\nSaved proposed trades → {out_path}")

    # Propose + lock
    lock_info = propose_trades(tickets, cadence=cadence)
    print("Lock Info:", lock_info)

    # --- Add reflection + RL eval + persistence ---
    try:
        etrade_trades = fetch_etrade_trade_history(account_id, count=100)
    except Exception:
        etrade_trades = []
    try:
        kraken_trades = fetch_kraken_trade_history(count=100)
    except Exception:
        kraken_trades = []

    all_trades = etrade_trades + kraken_trades
    trade_reflection = analyze_trade_history(all_trades, lookback_days=90)

    results = {
        "sleeve_targets": sleeve_targets,
        "portfolio_weights": portfolio,
        "sleeve_accounts": sleeve_w,
        "tickets": tickets,
        "probability": prob,
        "p50": p50,
        "run_dir": RUN_DIR,
        "trade_reflection": trade_reflection,
        "rl_evaluation": load_rl_eval_summary(),
        "rl_retrain_available": True
    }

    append_run_log(results)
    return results

# ================= Resilience Wrapper =================
def safe_run(func, *args, **kwargs):
    """
    Run engine/trainer functions safely.
    Returns {"status": "ok", "results": ...} or {"status": "error", "error": str}.
    Also logs errors to portfolio_log.json.
    """
    try:
        res = func(*args, **kwargs)
        return {"status": "ok", "results": res}
    except Exception as e:
        import traceback
        err_msg = f"{type(e).__name__}: {e}"
        tb = traceback.format_exc()
        error_payload = {"status": "error", "error": err_msg, "traceback": tb}
        try:
            append_run_log(error_payload)
        except Exception:
            pass
        print(f"❌ SafeRun caught error: {err_msg}")
        return error_payload

if __name__ == "__main__":
    results = main(cadence="weekly")
    print(json.dumps(results, indent=2, default=str))

ModuleNotFoundError: No module named 'RL_Portfolio_Trainer_v7_0'